# Step 4:
# Create Networks for all Individuals Based on Work, Home and Education

In [ ]:
import networkx as nx
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import collections
import csv

## 1, Load Data

In [ ]:
#Full pop
fpop = pd.read_csv('../results/Erie_pop_id.csv').iloc[:,1:]
fpop.head()

In [ ]:
len(fpop)

In [ ]:
fpop = fpop.set_index('id')
fpop.head()

## 2, Functions Creating Different types of Networks 

In [ ]:
def create_edges(x, g):
    if len(x) <= 5:
        sw = nx.complete_graph(len(x))
    else:
        sw = nx.newman_watts_strogatz_graph(len(x), 4, 0.3)
    sw = nx.relabel_nodes(sw, dict(zip(sw.nodes(), x.index.values)))# what's this line for?
    g.add_edges_from(sw.edges())

In [ ]:
def create_network(people, type):
    g = nx.Graph()
    g.add_nodes_from(people.index)
    if type == "school":
        grouped = people[people.age < 18].groupby('wp')
        grouped.apply(lambda x: create_edges(x, g))
    if type == 'work':
        grouped = people[people.age >= 18].groupby('wp')
        grouped.apply(lambda x: create_edges(x, g))
    if type == 'hhold':
        grouped = people.groupby('hhold')
        grouped.apply(lambda x: create_edges(x, g))
    return g

In [ ]:
def get_neighbors(x, g):
    return [n for n in g.neighbors(x)]

def to_csv(g, output_name):
    col1 = list(pop.index)
    col2 = pop.index.map(lambda x: get_neighbors(x, g))
    col2 = list(col2)
    for i in range(len(col2)):
        col2[i] = [col1[i]] + col2[i]
    with open(output_name + '.csv', 'w') as f:
        writer = csv.writer(f)
        writer.writerows(col2)

### 2.1 Daycare Network

In [ ]:
#Get daycare popualtion
dk = fpop[fpop.wp.str.contains('d')].copy()
print(len(dk))

In [ ]:
#### Create Daycare Network
print("creating daycare network...")
daycare = create_network(dk, "school")
print("the number of edges are: " + str(len(daycare.edges())))
print("done creating school network, writing to school graph")

In [ ]:
#Save to csv
print("converting school network to .csv")
pop = dk
to_csv(daycare, "../results/daycare_nw")
print("Done!")

In [ ]:
#Degree of Distribution
degree_sequence = sorted([d for n, d in daycare.degree()], reverse=True)  # degree sequence
#print ("Degree sequence", degree_sequence)
degreeCount = collections.Counter(degree_sequence)
deg, cnt = zip(*degreeCount.items())

In [ ]:
figure, ax = plt.subplots(figsize=(12, 8))

plt.bar(deg, cnt, width=0.80, color='#43a2ca')
plt.title("Daycare Network Degree Histogram", fontsize = 20, fontname = "Arial")
plt.ylabel("Frequency", fontsize = 16, fontname = "Arial")
plt.xlabel("Degree", fontsize = 16, fontname = "Arial")

ax.set_xticks([d for d in deg])
ax.set_xticklabels(deg)

rects = ax.patches

# For each bar: Place a label
for rect in rects:
    # Get X and Y placement of label from rect.
    y_value = rect.get_height()
    x_value = rect.get_x() + rect.get_width() / 2

    # Number of points between bar and label. Change to your liking.
    space = 5
    # Vertical alignment for positive values
    va = 'bottom'

    # If value of bar is negative: Place label below bar
    if y_value < 0:
        # Invert space to place label below
        space *= -1
        # Vertically align label at top
        va = 'top'

    # Use Y value as label and format number with one decimal place
    label = "{:.1f}".format(y_value)

    # Create annotation
    plt.annotate(
        label,                      # Use `label` as label
        (x_value, y_value),         # Place label at end of the bar
        xytext=(0, space),          # Vertically shift label by `space`
        textcoords="offset points", # Interpret `xytext` as offset in points
        ha='center',                # Horizontally center label
        va=va)                      # Vertically align label differently for
                                    # positive and negative values.

### 2.2, School Network

In [ ]:
#School Population
#kids = fpop[(fpop.age <=17) & (fpop.age >=4)].copy()
kids = fpop[fpop.wp.str.contains('s')].copy()
kids.shape

In [ ]:
print("creating school network...")
school = create_network(kids, "school")
print("the number of edges are: " + str(len(school.edges())))
print("done creating school network, writing to school graph")

In [ ]:
#Save to csv
print("converting school network to .csv")
pop = kids
to_csv(school, "../results/school_nw")
print("Done!")

In [ ]:
#Not Run
print('Saving 1...')
nx.write_graphml(school, 'ERIE_Results/network/school/school.graphml')
print('Saving 2...')
nx.write_gexf(school, 'ERIE_Results/network/school/school.gexf')
print('Saving 3...')
nx.write_edgelist(school, 'ERIE_Results/network/school/school.edgelist')
print('Done...')

In [ ]:
degree_sequence = sorted([d for n, d in school.degree()], reverse=True)  # degree sequence
#print ("Degree sequence", degree_sequence)
degreeCount = collections.Counter(degree_sequence)
deg, cnt = zip(*degreeCount.items())

In [ ]:
figure, ax = plt.subplots(figsize=(12, 8))

plt.bar(deg, cnt, width=0.80, color='#43a2ca')
plt.title("School Network Degree Histogram", fontsize = 20, fontname = "Arial")
plt.ylabel("Frequency", fontsize = 16, fontname = "Arial")
plt.xlabel("Degree", fontsize = 16, fontname = "Arial")

ax.set_xticks([d for d in deg])
ax.set_xticklabels(deg)

rects = ax.patches

# For each bar: Place a label
for rect in rects:
    # Get X and Y placement of label from rect.
    y_value = rect.get_height()
    x_value = rect.get_x() + rect.get_width() / 2

    # Number of points between bar and label. Change to your liking.
    space = 5
    # Vertical alignment for positive values
    va = 'bottom'

    # If value of bar is negative: Place label below bar
    if y_value < 0:
        # Invert space to place label below
        space *= -1
        # Vertically align label at top
        va = 'top'

    # Use Y value as label and format number with one decimal place
    label = "{:.1f}".format(y_value)

    # Create annotation
    plt.annotate(
        label,                      # Use `label` as label
        (x_value, y_value),         # Place label at end of the bar
        xytext=(0, space),          # Vertically shift label by `space`
        textcoords="offset points", # Interpret `xytext` as offset in points
        ha='center',                # Horizontally center label
        va=va)                      # Vertically align label differently for
                                    # positive and negative values.

### 2.3  Work Network, people work in the workplace

In [ ]:
adults = fpop[fpop.age>=18].copy()

In [ ]:
adults.shape

In [ ]:
#select people
ad_work = adults[adults.wp.str.contains("w")]

In [ ]:
#People Commute
ad_work.shape

In [ ]:
#Workplaces Networks
print("Creating workplace network...")
work = create_network(ad_work, "work")
print("The number of edges are: " + str(len(work.edges())))
print("Done creating workplace network, writing to work graph")

In [ ]:
nx.write_graphml(work, 'ERIE_Results/network/work/work.graphml')
nx.write_gexf(work, 'ERIE_Results/network/work/work.gexf')
nx.write_edgelist(work, 'ERIE_Results/network/work/work.edgelist')

In [ ]:
print("converting workplace network to .csv")
pop = ad_work
to_csv(work, "../results/work_nw")
print("Done!")

In [ ]:
degree_sequence = sorted([d for n, d in work.degree()], reverse=True)  # degree sequence
#print ("Degree sequence", degree_sequence)
degreeCount = collections.Counter(degree_sequence)
deg, cnt = zip(*degreeCount.items())

In [ ]:
figure, ax = plt.subplots(figsize=(12, 8))

plt.bar(deg, cnt, width=0.80, color='#43a2ca')
plt.title("Workplaces Network Degree Histogram", fontsize = 20, fontname = "Arial")
plt.ylabel("Frequency", fontsize = 16, fontname = "Arial")
plt.xlabel("Degree", fontsize = 16, fontname = "Arial")

ax.set_xticks([d for d in deg])
ax.set_xticklabels(deg)

rects = ax.patches

# For each bar: Place a label
for rect in rects:
    # Get X and Y placement of label from rect.
    y_value = rect.get_height()
    x_value = rect.get_x() + rect.get_width() / 2
    # Number of points between bar and label. Change to your liking.
    space = 5
    # Vertical alignment for positive values
    va = 'bottom'

    # If value of bar is negative: Place label below bar
    if y_value < 0:
        # Invert space to place label below
        space *= -1
        # Vertically align label at top
        va = 'top'

    # Use Y value as label and format number with one decimal place
    label = "{:.1f}".format(y_value)

    # Create annotation
    plt.annotate(
        label,                      # Use `label` as label
        (x_value, y_value),         # Place label at end of the bar
        xytext=(0, space),          # Vertically shift label by `space`
        textcoords="offset points", # Interpret `xytext` as offset in points
        ha='center',                # Horizontally center label
        va=va)                      # Vertically align label differently for
                                    # positive and negative values.

### 2.4 Household Network

In [ ]:
fpop.shape

In [ ]:
fpop.head()

In [ ]:
print("creating hhold network...")
hhold = create_network(fpop, "hhold")
print("the number of edges are: " + str(len(hhold.edges())))
print("Done creating school network, writing to hhold graph")

In [ ]:
print('S1..')
nx.write_graphml(hhold, 'ERIE_Results/network/household/hhold.graphml')
print('S2..')
nx.write_gexf(hhold, 'ERIE_Results/network/household/hhold.gexf')
print('S3..')
nx.write_edgelist(hhold, 'ERIE_Results/network/household/hhold.edgelist')
print("Done!")

In [ ]:
print("converting hhold network to .csv")
pop = fpop
to_csv(hhold, "../results/hhold_nw")
print("Done!")

In [ ]:
degree_sequence = sorted([d for n, d in hhold.degree()], reverse=True)  # degree sequence
#print ("Degree sequence", degree_sequence)
degreeCount = collections.Counter(degree_sequence)
deg, cnt = zip(*degreeCount.items())

In [ ]:
figure, ax = plt.subplots(figsize=(12, 8))

plt.bar(deg, cnt, width=0.80, color='#43a2ca')
plt.title("Households Network Degree Histogram", fontsize = 20, fontname = "Arial")
plt.ylabel("Frequency", fontsize = 16, fontname = "Arial")
plt.xlabel("Degree", fontsize = 16, fontname = "Arial")

ax.set_xticks([d for d in deg])
ax.set_xticklabels(deg)

rects = ax.patches

# For each bar: Place a label
for rect in rects:
    # Get X and Y placement of label from rect.
    y_value = rect.get_height()
    x_value = rect.get_x() + rect.get_width() / 2

    # Number of points between bar and label. Change to your liking.
    space = 5
    # Vertical alignment for positive values
    va = 'bottom'

    # If value of bar is negative: Place label below bar
    if y_value < 0:
        # Invert space to place label below
        space *= -1
        # Vertically align label at top
        va = 'top'

    # Use Y value as label and format number with one decimal place
    label = "{:.1f}".format(y_value)

    # Create annotation
    plt.annotate(
        label,                      # Use `label` as label
        (x_value, y_value),         # Place label at end of the bar
        xytext=(0, space),          # Vertically shift label by `space`
        textcoords="offset points", # Interpret `xytext` as offset in points
        ha='center',                # Horizontally center label
        va=va)                      # Vertically align label differently for
                                    # positive and negative values.